In [3]:
%run ./0623-experiment-notebook.ipynb

/Users/bgulowaty/studia/projekty/scalarizing/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-16 17:52:07,375	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from mlutils.mlflow.utils import create_runs_for_params, get_unfinished_run_ids, get_runs, get_unfinished_runs, get_run_params, experiment_name_to_id
from glob import glob
from mlutils.distributedcomputing.experiments import run_experiments
from sklearn.model_selection import ParameterGrid

In [4]:
all_processed_data_files = glob(f"../../datasets/notebooks/processed/*.csv")

all_file_names = [
    file.split('/')[-1].split('train')[0].rstrip('-') for file in all_processed_data_files if 'tra.csv' in file
]
print(all_file_names[:5])

['shuttle-5-5tra.csv', 'pima-5-5tra.csv', 'wisconsin-5-4tra.csv', 'automobile-5-2tra.csv', 'spambase-5-1tra.csv']


In [5]:
train_file_paths = [it for it in all_processed_data_files if 'tra.csv' in it]
dataset_names = {path.split('/')[-1].split('-')[0] for path in train_file_paths}
selected_datasets = []
for name in dataset_names:
    selected_datasets.append([p for p in train_file_paths if name in p][0])

In [6]:
params_test = ParameterGrid({
    'pop_size': [5],
    'n_gen': [25],
    'ensemble_size': [50, 100],
    'final_ensemble_size': [5, 10, 30],
    'train_path': train_file_paths
})

In [7]:
params_test_ids = create_runs_for_params(params_test, experiment_name='v2-withCache')

2023-08-16 17:52:16.697 | INFO     | mlutils.mlflow.utils:create_runs_for_params:31 - Experiment not found, creating new. Id=447219576146047241
100%|██████████| 600/600 [00:02<00:00, 211.30it/s]


In [8]:
the_experiment = get_runs(experiment_name_to_id('v2-withCache'))

In [23]:
the_experiment.query("status != 'RUNNING'")

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.train_path,params.n_gen,params.ensemble_size,params.pop_size,params.final_ensemble_size,tags.mlflow.runName
598,55f96ab062e543d79746aecbb6fa540f,386990130301449091,FINISHED,file:///Users/bgulowaty/studia/projekty/scalar...,2023-08-15 19:28:07.057000+00:00,2023-08-16 00:27:41.565000+00:00,0.027926,../../datasets/notebooks/processed/wine-5-2tra...,100,50,100,5,rogue-newt-318


In [9]:
unfinished_run_ids = get_unfinished_run_ids(experiment_name_to_id("v2-withCache"))

In [10]:
unfinished_run_ids[:3]

['0dab767811fe4e61af211b43731ec32d',
 '4993548b62684dbb91291b24099607fe',
 '51da72e2ecd248069d08b7cf914b48b6']

In [ ]:
%%prun -s cumulative
run_experiments(unfinished_run_ids[:3], do_experiment, backend='plain')

2023-08-16 08:01:35.772 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:11 - {}
2023-08-16 08:01:35.780 | INFO     | __main__:do_experiment:4 - {'train_path': '../../datasets/notebooks/processed/banana-5-3tra.csv', 'final_ensemble_size': '30', 'n_gen': '100', 'pop_size': '100', 'ensemble_size': '100'}
/Users/bgulowaty/studia/projekty/scalarizing/.venv/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/bgulowaty/studia/projekty/scalarizing/.venv/lib/python3.8/site-packages/mlxtend/classifier/ensemble_vote.py:173: UserWarning: fit_base_estimators=False enforces use_clones to be `False`
  warnings.warn(
/Users/bgulowaty/studia/projekty/scalarizing/.venv/lib/python3.8/site-packages/mlxtend/classifier/ensemble_vote.py:173: UserWarning: fit_base_estimators=False enforces use_clones to be `False`
  warnings.warn(
/Users/bgulowaty/studia/pr

In [ ]:
run_experiments(unfinished_run_ids, 'do_experiment', notebook_path="0623-experiment-notebook.ipynb", backend='papermill', n_jobs=6)

2023-08-16 17:52:48.194 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:11 - {'notebook_path': '0623-experiment-notebook.ipynb', 'n_jobs': 6}
2023-08-16 17:52:48.194 | WARNING  | mlutils.distributedcomputing.experiments:run_experiments:21 - Using Papermill backend. Make sure you execute this function not from experiment notebook!
2023-08-16 17:52:48.195 | INFO     | mlutils.distributedcomputing.job_scheduling:create_experiment_notebook:91 - 
    Creating experiment notebook
        notebook_name=0623-experiment-notebook.ipynb
        experiment_function=do_experiment
        instance_id_param_name=EXPERIMENT_INSTANCE_ID
        
2023-08-16 17:52:48.215 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:34 - Notebook path=/Users/bgulowaty/studia/projekty/scalarizing/notebooks/0623-experiment-notebook-89cd86cf-261c-4392-ab56-1da5b1dd73bb.ipynb
2023-08-16 17:52:48.296 | INFO     | mlutils.distributedcomputing.job_scheduling:run_in_papermill:37 - 
 

In [538]:
%timeit
clf_cached.predict_proba([[1,2], [5,0], [23,32], [1,2], [5,0], [23,32], [1,2], [5,0], [23,32], [1,2], [5,0], [23,32], [1,2], [5,0], [23,32], [1,2], [5,0], [23,32], [1,2], [5,0], [23,32]])

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [494]:
np.array([0, 1, 2]).reshape(3, 1)

array([[0],
       [1],
       [2]])

In [233]:
def do_hash(numpy_array):
    return str(numpy_array.tobytes())

In [234]:
hashes = np.apply_along_axis(do_hash, axis=1, arr=x)

In [227]:
from cachetools import LRUCache
cache = LRUCache(maxsize=512)
cache[hashes[0]] = 1
cache[hashes[2]] = 2

In [87]:
#hashes = np.apply_along_axis(lambda it: it.tobytes(), axis=1, arr=x)

In [228]:
already_cached = np.in1d(hashes, cache)
indicies_cached = np.where(already_cached)[0]
indicies_not_cached = np.where(~already_cached)[0]



In [241]:
def do_classify(val):
    return 5

In [264]:
values_from_cache = [cache[it] for it in hashes[already_cached]]
values_classified = [do_classify(it) for it in x[indicies_not_cached]]

array([[0., 0.],
       [0., 0.]])

In [496]:
y_soft = np.array([[0.1, 0.2], [5, 6], [1,1]])
y_classified = np.zeros((4, 2))
y_classified

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [358]:
np.array([[0], [2]])

(2, 1)

In [498]:
indicies_cached = np.array([0, 1, 3])

In [502]:
y_classified

array([[0.1, 0.2],
       [5. , 6. ],
       [0. , 0. ],
       [1. , 1. ]])

In [501]:
np.put_along_axis(y_classified, indicies_cached.reshape((3, 1)), y_soft, axis=0)
# np.put_along_axis(y_classified, indicies_not_cached, values_classified, axis=0)

In [350]:
np.put_along_axis(y_classified, np.array([[0], [2]]), y_soft, axis=0)
# np.put_along_axis(y_classified, indicies_not_cached, values_classified, axis=0)

In [361]:
y_classified

array([[0.1, 0.2],
       [0. , 0. ],
       [5. , 6. ],
       [0. , 0. ]])

In [340]:
y_classified

array([[0.1, 0.2],
       [5. , 6. ],
       [5. , 6. ],
       [0. , 0. ]])

In [261]:
np.apply_along_axis(lambda it: cache[it], axis=1, arr=hashes[already_cached])

AxisError: axis 1 is out of bounds for array of dimension 1

In [245]:
np.piecewise(x, [already_cached, ~already_cached], [lambda it: cache[do_hash(it)], do_classify])

KeyError: "b'\\x01\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x02\\x00\\x00\\x00\\x00\\x00\\x00\\x00L\\x01\\x00\\x00\\x00\\x00\\x00\\x00\\xa8\\x01\\x00\\x00\\x00\\x00\\x00\\x00'"

In [229]:
indicies_cached

array([0, 2])

In [236]:
hashes_cached[0] == do_hash(x[0])

True

In [61]:
to_evaluate_by_clf = x[indicies_not_cached]

to_get_from_cache

In [62]:
to_evaluate_by_clf

array([[332, 424],
       [  1,   1]])

In [59]:
indicies_cached

array([0, 1])

In [46]:
not_cached = ~already_cached

In [49]:
x[not_cached]

array([[332, 424],
       [  1,   1]])

In [13]:
cache[x[0].tobytes()]

5

In [8]:
from cachetools import LRUCache
cache = LRUCache(maxsize=512)
cache['abc'] = 'sda'
'sda' in cache.keys()

False

In [18]:
experiment_scalarizing[0]

NameError: name 'experiment_scalarizing' is not defined